# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [57]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import geopandas as gpd
from shapely.geometry import Point

# Import API key
from api_keys import geoapify_key

In [58]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
file_path = r"C:\Users\Qazi Fabia Hoq\OneDrive\Desktop\python-api-challenge\output_data\cities.csv"
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadibu,12.6500,54.0333,26.34,69,99,8.62,YE,1701583154
1,1,waitangi,-43.9535,-176.5597,15.46,91,100,4.02,NZ,1701583154
2,2,jamaame,0.0697,42.7450,30.27,61,32,5.20,SO,1701583154
3,3,whitehorse,60.7161,-135.0538,-10.57,92,100,2.57,CA,1701583155
4,4,khanty-mansiysk,61.0042,69.0019,-14.94,72,40,3.00,RU,1701583155


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [59]:
# Configure the map plot
map_plot = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=700,
    frame_height=500,
    size='Humidity',
    color='Humidity',
    cmap='YlOrRd',  # Specify the colormap
    alpha=0.7,
    hover_cols=['City', 'Humidity'],  # Columns to display in the hover tooltip
)

# Display the map
map_plot


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

In [60]:
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hadibu,12.6500,54.0333,26.34,69,99,8.62,YE,1701583154
1,1,waitangi,-43.9535,-176.5597,15.46,91,100,4.02,NZ,1701583154
2,2,jamaame,0.0697,42.7450,30.27,61,32,5.20,SO,1701583154
3,3,whitehorse,60.7161,-135.0538,-10.57,92,100,2.57,CA,1701583155
4,4,khanty-mansiysk,61.0042,69.0019,-14.94,72,40,3.00,RU,1701583155


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [61]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_conditions = (
    (city_data_df['Max Temp'] > 21) & 
    (city_data_df['Max Temp'] < 27) & 
    (city_data_df['Wind Speed'] < 4.5) & 
    (city_data_df['Cloudiness'] == 0)
)

# Filter the DataFrame
ideal_cities_df = city_data_df[ideal_weather_conditions]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
59,59,sur,22.5667,59.5289,26.04,51,0,2.25,OM,1701583164
305,305,merritt island,28.5392,-80.6720,24.49,94,0,2.57,US,1701582933
417,417,birendranagar,28.6019,81.6339,24.71,23,0,1.10,NP,1701583257
423,423,samdrup jongkhar,26.9167,91.6167,23.81,32,0,2.43,BT,1701583258
531,531,cabo san lucas,22.8909,-109.9124,22.51,67,0,1.53,MX,1701583229
539,539,qeshm,26.9581,56.2719,25.93,57,0,0.56,IR,1701583289


### Step 3: Create a new DataFrame called `hotel_df`.

In [62]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API

hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()


,City,Country,Lat,Lng,Humidity,Hotel Name
59,sur,OM,22.5667,59.5289,51,
305,merritt island,US,28.5392,-80.6720,94,
417,birendranagar,NP,28.6019,81.6339,23,
423,samdrup jongkhar,BT,26.9167,91.6167,32,
531,cabo san lucas,MX,22.8909,-109.9124,67,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [63]:
 # Define the API parameters
params = {
    "apiKey":geoapify_key,
    "format":"json"
}

# Set the base URL
base_url = "https://api.geoapify.com/v1/geocode/search"

In [65]:
# Set parameters to search for both airport and hotel
radius = 10000  # Set your desired search radius
params = {
     "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 50  
}

# Print a message to follow up the searches
print("Starting searches")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat, lng = row['Lat'], row['Lng']  # Correct column names
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    response = response.json()
    
    # Grab the first place from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no place is found, set the hotel name as "No place found".
        hotel_df.loc[index, "Hotel Name"] = "No place found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest place: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df


Starting searches
sur - nearest place: Sur Hotel
merritt island - nearest place: No place found
birendranagar - nearest place: Aacharya
samdrup jongkhar - nearest place: No place found
cabo san lucas - nearest place: Comfort Rooms
qeshm - nearest place: هتل آپادانا


,City,Country,Lat,Lng,Humidity,Hotel Name
59,sur,OM,22.5667,59.5289,51,Sur Hotel
305,merritt island,US,28.5392,-80.6720,94,No place found
417,birendranagar,NP,28.6019,81.6339,23,Aacharya
423,samdrup jongkhar,BT,26.9167,91.6167,32,No place found
531,cabo san lucas,MX,22.8909,-109.9124,67,Comfort Rooms
539,qeshm,IR,26.9581,56.2719,57,هتل آپادانا


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [66]:
# Configure the map plot with additional hover information
map_plot_with_hotel = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo=True,
    tiles='OSM',
    frame_width=700,
    frame_height=500,
    size='Humidity',
    color='Humidity',
    cmap='YlOrRd',  # Specify the colormap
    alpha=0.7,
    hover_cols=['City', 'Humidity', 'Hotel Name'],  # Columns to display in the hover tooltip
)

# Display the map
map_plot_with_hotel


:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Hotel Name)